In [1]:
import pandas as pd

In [2]:
%cd  ../

d:\Personal\project_temp\ReliSource\1st_phase\ReliSource


In [3]:
train_data =  pd.read_csv('data/train.csv')
test_data =  pd.read_csv('data/test.csv')
train_data

,UID,col_0,col_1,col_2,col_3,col_4,col_5,col_6,y
0,0,A 0,B0,C2,D1,100,E1,F2,237000
1,1,A1,B0,C11,D4,100,E4,F2,86193
2,2,A0,B0,C18,D0,0,E0,F2,169200
3,3,A2,B0,C11,D1,100,E1,F2,58000
4,4,A0,B0,C67,D1,0,E1,F2,235000
...,...,...,...,...,...,...,...,...,...
2623,2623,A1,B0,C2,D1,0,E1,F2,102100
2624,2624,A0,B0,C8,D1,0,E1,F2,129300
2625,2625,A0,BO,C7,D1,100,E1,F2,275300
2626,2626,A0,B0,C11,D1,100,E1,F2,150000


In [4]:
train_data = train_data.drop_duplicates()

In [5]:
# Function to check if a value is a string
def is_string(value):
    return isinstance(value, str)

# Function to check if a string contains spaces
def contains_spaces(s):
    return ' ' in s

# Loop through all columns and remove trailing whitespace for string columns
# Remove spaces between words in the 'TextColumn'
# Function to remove spaces between words and remove trailing whitespace for object type columns
def rm_spaces(dataframe):
    for column in dataframe.columns:
        if dataframe[column].dtype == 'object':  # Check if the column contains strings
            dataframe[column] = dataframe[column].str.strip()
            dataframe[column] = dataframe[column].str.replace(' ', '')

    return dataframe


def count_string_and_non_string_values(data_frame):

    # Get a list of column names with object (string) data type
    string_columns = data_frame.select_dtypes(include=['object']).columns

    # Initialize lists to store non-string values and their locations
    non_string_values = []
    non_string_locations = []

    # Apply the function to each element in the DataFrame, count the results, and collect non-string values
    string_count = data_frame[string_columns].applymap(is_string).sum().sum()
    for col in string_columns:
        non_string_col = data_frame[col][data_frame[col].apply(lambda x: not is_string(x))]
        non_string_values.extend(non_string_col)
        non_string_locations.extend([(col, idx) for idx in non_string_col.index])

    # Return the counts and non-string values
    return {
        'Count of string values': string_count,
        'Count of non-string values': len(non_string_values),
        'Non-string values': non_string_values,
        'Non-string value locations': non_string_locations
    }

def status_of_space(dataframe):
    # Get a list of column names with object (string) data type
    string_columns = dataframe.select_dtypes(include=['object']).columns


    # Apply the function to each element in the DataFrame and count the results
    string_count = dataframe[string_columns].applymap(is_string).sum().sum()
    non_string_count = dataframe[string_columns].applymap(lambda x: not is_string(x)).sum().sum()

    # Print the counts
    print(f'Count of string values: {string_count}')
    print(f'Count of non-string values: {non_string_count}')

    # Convert mixed columns to string columns
    strdata = dataframe[string_columns].astype(str)

    # Apply the function to each element in the selected columns and sum the results
    count_with_spaces = strdata[string_columns].applymap(contains_spaces).sum().sum()

    # Print the count of strings with spaces
    print(f'Count of strings with spaces: {count_with_spaces}')

def replace_null_with_missing(dataframe):
    return dataframe.fillna("Missing")

def replace_null_with_number(dataframe):
    return dataframe.fillna("-1")

def replace_values_in_columns(df, replacements):
    """
    Replace specified values in the specified columns of a DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame.
    replacements (dict): A dictionary where keys are column names and values are dictionaries
                         specifying old and new values to replace.

    Returns:
    DataFrame: The DataFrame with values replaced in specified columns.
    """
    for column, replace_dict in replacements.items():
        old_value = replace_dict.get('old_value', None)
        new_value = replace_dict.get('new_value', None)
        if old_value is not None and new_value is not None:
            df[column] = df[column].replace(old_value, new_value)
    return df

def split_columns_into_parts(df, columns_to_split):
    """
    Split specified columns into their numerical and string parts in a DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame.
    columns_to_split (list): A list of column names to split.

    Returns:
    DataFrame: The DataFrame with split columns.
    """
    for col in columns_to_split:
        num_part, str_part = zip(*df[col].astype(str).apply(lambda x: ("".join(filter(str.isdigit, x)), "".join(filter(str.isalpha, x)))))
        df[f'{col}_numerical'] = num_part
        df[f'{col}_string'] = str_part
        df.drop(columns=[col], inplace=True)
    
    return df

def remove_string_columns(dataframe):
    dataframe = dataframe.drop(columns=[x for x in dataframe.columns if "string" in x])
    return dataframe

def fixed_data_types(dataframe):
    for c in dataframe.columns:
        dataframe[c] = dataframe[c].astype(int)
    return dataframe

def dataset_processing(dataframe):
    dataframe = rm_spaces(dataframe)
    # dataframe = replace_null_with_missing(dataframe)
    dataframe = replace_null_with_number(dataframe)

    replacements = {
    'col_0': {'old_value': 'AO', 'new_value': 'A0'},
    'col_1': {'old_value': 'BO', 'new_value': 'B0'},
    'col_6': {'old_value': 'FO', 'new_value': 'F0'}
    }
    dataframe = replace_values_in_columns(dataframe, replacements)
    columns_to_split = ['col_0', 'col_1', 'col_2', 'col_3', 'col_5', 'col_6']
    dataframe = split_columns_into_parts(dataframe, columns_to_split)
    dataframe = remove_string_columns(dataframe)
    dataframe = fixed_data_types(dataframe)
    return dataframe

In [6]:
# processed_test_data = dataset_processing(test_data)
# processed_test_data.info()
# processed_test_data.head(60)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   UID              1127 non-null   int32
 1   col_4            1127 non-null   int32
 2   col_0_numerical  1127 non-null   int32
 3   col_1_numerical  1127 non-null   int32
 4   col_2_numerical  1127 non-null   int32
 5   col_3_numerical  1127 non-null   int32
 6   col_5_numerical  1127 non-null   int32
 7   col_6_numerical  1127 non-null   int32
dtypes: int32(8)
memory usage: 35.3 KB


,UID,col_4,col_0_numerical,col_1_numerical,col_2_numerical,col_3_numerical,col_5_numerical,col_6_numerical
0,2628,100,0,0,11,1,1,2
1,2629,0,0,0,11,1,1,2
2,2630,100,0,0,11,1,1,2
3,2631,100,0,0,26,1,1,0
4,2632,100,0,0,11,1,1,1
5,2633,100,0,0,2,1,1,2
6,2634,0,0,0,11,1,1,2
7,2635,50,1,1,2,12,11,2
8,2636,0,0,0,11,1,1,0
9,2637,0,0,0,9,1,1,2


In [6]:
processed_train_data = dataset_processing(train_data)
processed_test_data = dataset_processing(test_data)

In [7]:
processed_train_data

,UID,col_4,y,col_0_numerical,col_0_string,col_1_numerical,col_1_string,col_2_numerical,col_2_string,col_3_numerical,col_3_string,col_5_numerical,col_5_string,col_6_numerical,col_6_string
0,0,100,237000,0,A,0,B,2,C,1,D,1,E,2,F
1,1,100,86193,1,A,0,B,11,C,4,D,4,E,2,F
2,2,0,169200,0,A,0,B,18,C,0,D,0,E,2,F
3,3,100,58000,2,A,0,B,11,C,1,D,1,E,2,F
4,4,0,235000,0,A,0,B,67,C,1,D,1,E,2,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,2623,0,102100,1,A,0,B,2,C,1,D,1,E,2,F
2624,2624,0,129300,0,A,0,B,8,C,1,D,1,E,2,F
2625,2625,100,275300,0,A,0,B,7,C,1,D,1,E,2,F
2626,2626,100,150000,0,A,0,B,11,C,1,D,1,E,2,F


In [8]:
processed_train_data.columns

Index(['UID', 'col_4', 'y', 'col_0_numerical', 'col_0_string',
       'col_1_numerical', 'col_1_string', 'col_2_numerical', 'col_2_string',
       'col_3_numerical', 'col_3_string', 'col_5_numerical', 'col_5_string',
       'col_6_numerical', 'col_6_string'],
      dtype='object')

In [9]:
proc2 = processed_train_data.drop(columns=[x for x in processed_train_data.columns if "string" in x])


In [10]:
proc2 = rm_spaces(proc2)

In [ ]:
# def proces_empty_string_value(dataframe):
#     for c in dataframe.columns:
#         dataframe.loc[dataframe[c] == '', c] = "-1"
#     return dataframe

In [11]:
for c in proc2.columns:
    proc2.loc[proc2[c] == '', c] = "-1"

In [12]:
for c in proc2.columns:
    proc2[c] = proc2[c].astype(int)

In [16]:
for c in proc2.columns:
    if c != "y" and  c != "UID":
        print("Printing for: ",c,"\n",proc2[c].value_counts())

Printing for:  col_4 
 col_4
0      1343
100    1149
50      136
Name: count, dtype: int64
Printing for:  col_0_numerical 
 col_0_numerical
 0    1679
 1     542
 2     195
-1     139
 3      73
Name: count, dtype: int64
Printing for:  col_1_numerical 
 col_1_numerical
 0    2518
-1      86
 3      11
 2       7
 1       6
Name: count, dtype: int64
Printing for:  col_2_numerical 
 col_2_numerical
11    728
2     587
4     416
9     204
15     77
     ... 
62      1
63      1
92      1
76      1
66      1
Name: count, Length: 90, dtype: int64
Printing for:  col_3_numerical 
 col_3_numerical
1     2093
4      116
0       61
2       60
6       54
      ... 
32       1
71       1
75       1
66       1
73       1
Name: count, Length: 68, dtype: int64
Printing for:  col_5_numerical 
 col_5_numerical
1     2120
4      121
2       62
0       57
6       44
      ... 
45       1
47       1
20       1
67       1
62       1
Name: count, Length: 67, dtype: int64
Printing for:  col_6_numerical 
 col

In [24]:
print(proc2["col_6_numerical"].value_counts())

col_6_numerical
 2    2104
 0     309
-1     112
 1     103
Name: count, dtype: int64


In [15]:
proc2.head(60)

,UID,col_4,y,col_0_numerical,col_1_numerical,col_2_numerical,col_3_numerical,col_5_numerical,col_6_numerical
0,0,100,237000,0,0,2,1,1,2
1,1,100,86193,1,0,11,4,4,2
2,2,0,169200,0,0,18,0,0,2
3,3,100,58000,2,0,11,1,1,2
4,4,0,235000,0,0,67,1,1,2
5,5,100,170000,0,0,11,1,1,2
6,6,100,183600,0,0,11,1,1,0
7,7,0,100000,0,0,4,1,1,2
8,8,100,175000,0,0,2,1,1,2
9,9,100,136000,0,0,2,1,1,2


In [26]:
proc2.to_csv("data/cleaned_train_iffat.csv",index=False)

In [7]:
processed_test_data.to_csv("data/cleaned_test_tonmoy.csv",index=False)